In [1]:
# Libraries import 
import pdbufr
import sys
import traceback
 
from math import isnan

from eccodes import *
from ecmwf.opendata import Client
from math import isnan
from datetime import datetime, timedelta

import os
import birdy
import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr
import requests

from ipywidgets import interact
import ipyleaflet
import ipywidgets as widgets

from Magics import macro as magics
from IPython.display import display
from Magics.macro import *

import warnings
warnings.filterwarnings("ignore")

from utils import *

In [2]:
## Widget to define the starting date of the forecast of which we want to download the data ##

start_date_forecast = widgets.DatePicker(
    description = 'Forecast date:',
    value = datetime(datetime.now().year, datetime.now().month, datetime.now().day, hour=0),
)

start_date_forecast.style.description_width = '90px'

# Print widget
start_date_forecast

DatePicker(value=datetime.datetime(2023, 8, 23, 0, 0), description='Forecast date:', step=1, style=Description…

In [3]:
## Download the tracks forecast based on the starting date selected ##
# If too early in the morning the forecast of the current day cannot be donwloaded, need to fix this -> If raiseError: download forecast of the previous day
    
download_tracks_forecast(start_date_forecast.value)

## Donwload the observed data from IBTrACS##

url = 'https://www.ncei.noaa.gov/data/international-best-track-archive-for-climate-stewardship-ibtracs/v04r00/access/csv/ibtracs.ACTIVE.list.v04r00.csv'
r = requests.get(url, allow_redirects=True)
save_file = 'data/ibtracs.ACTIVE.list.v04r00.csv'
with open(save_file, 'wb') as f:
    f.write(r.content)

20230823000000-240h-enfo-tf.bufr:   0%|          | 0.00/913k [00:00<?, ?B/s]

In [4]:
## Load forecasted tracks data in a dataframe ##
# create_storms_df loads a file called tc_test_track_data.bufr
df_storms_forecast = create_storms_df()

## Load observed tracks data in a dataframe ## 
df_storms_observed = pd.read_csv('data/ibtracs.ACTIVE.list.v04r00.csv', header=[0,1])

## Find the correspondent cyclone across forecast and observations ## 
storms_pair = storms_pairing(df_storms_forecast, df_storms_observed)

In [5]:
# Cyclone dropdown selection widget to select cyclone of which to plot the data
cyclone = widgets.Dropdown(
    options = storms_pair,
    description = 'Active Storms:',
    disabled=False,
)
cyclone.style.description_width = '90px'

# Update storms list when downloading new tracks data
def update_storms_list(_):
    download_tracks_forecast(start_date_forecast.value)
    df_storms = create_storms_df()
    df_storms_observed = pd.read_csv('data/ibtracs.ACTIVE.list.v04r00.csv', header=[0,1])
    storms_pair = storms_pairing(df_storms_forecast, df_storms_observed)
    cyclone.options = storms_pair.tolist()

start_date_forecast.observe(update_storms_list, names='value')

# Print widget
display(cyclone)

In [7]:
## UPDATE BUTTON TO DOWNLOAD NEW DATA AND CREATE STORM DATAFRAME DEPENDING ON THE CYCLONE SELECTED ##

update_button = widgets.Button(
    description = 'Update Forecast',
    style=dict(
        button_color='blue',
        font_weight='bold',
        text_color='lightgreen',
        text_decoration='underline',
))

update_output = widgets.Output()

# Download new forecast data when clicking the Update Forecast Button
def on_button_clicked(b):
    with update_output:
        df_storms = create_storms_df()
        df_storms_observed = pd.read_csv('data/ibtracs.ACTIVE.list.v04r00.csv', header=[0,1])
        plot_cyclone_tracks_ipyleaflet(cyclone.value)
        print("Forecast updated!", end='\r')

update_button.on_click(on_button_clicked)

# Print widget
display(update_button, update_output)

Button(description='Update Forecast', style=ButtonStyle(button_color='blue', font_weight='bold', text_color='l…

Output()